<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Data Reshaping in Python


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [2]:
import pandas as pd
import os

dengue = pd.read_csv(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue.csv"))

ParserError: Error tokenizing data. C error: Expected 14 fields in line 87871, saw 16


In [ ]:
dengue = pd.read_csv(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue.csv"),on_bad_lines='warn')

Pandas offers **on_bad_lines='warn'** to let you know if something does not make sense. As you see, you have 8 lines that were omitted. This is what you have now:

In [ ]:
dengue.shape

You can try in a different way:

In [ ]:
dengue2=pd.read_table(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue.csv"))
dengue2

You did not get a warning, and in fact you got 8 more rows. You can try to identify what is wrong:

In [ ]:
dengue2.iloc[87867:87873,0]

In [ ]:
# use that weird text:
dengue2[dengue2.iloc[:,0].str.contains("I\\,II",regex=False)]

I have prepared a cleaner version:

In [3]:
dengue = pd.read_csv(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue_ok.csv"))
dengue.info()

/var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/ipykernel_33850/4080489231.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dengue = pd.read_csv(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue_ok.csv"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501692 entries, 0 to 501691
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   departamento  501692 non-null  object 
 1   provincia     501692 non-null  object 
 2   distrito      501692 non-null  object 
 3   localidad     400753 non-null  object 
 4   enfermedad    501692 non-null  object 
 5   ano           501692 non-null  int64  
 6   semana        501692 non-null  int64  
 7   diagnostic    501692 non-null  object 
 8   diresa        501669 non-null  float64
 9   ubigeo        501692 non-null  int64  
 10  localcod      407822 non-null  object 
 11  edad          501692 non-null  int64  
 12  tipo_edad     501692 non-null  object 
 13  sexo          501692 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 53.6+ MB


In [4]:
dengue = pd.read_csv(os.path.join('FilesToReshape' , "datos_abiertos_vigilancia_dengue_ok.csv"),encoding='utf-8', engine='python')
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501692 entries, 0 to 501691
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   departamento  501692 non-null  object 
 1   provincia     501692 non-null  object 
 2   distrito      501692 non-null  object 
 3   localidad     400753 non-null  object 
 4   enfermedad    501692 non-null  object 
 5   ano           501692 non-null  int64  
 6   semana        501692 non-null  int64  
 7   diagnostic    501692 non-null  object 
 8   diresa        501669 non-null  float64
 9   ubigeo        501692 non-null  int64  
 10  localcod      407822 non-null  object 
 11  edad          501692 non-null  int64  
 12  tipo_edad     501692 non-null  object 
 13  sexo          501692 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 53.6+ MB


Let me select a subset of columns:

In [19]:
toSelect=['departamento', 'provincia', 'distrito','ano', 'semana', 'sexo','enfermedad']
dengueSub=dengue[toSelect]
dengueSub.head()

,departamento,provincia,distrito,ano,semana,sexo,enfermedad
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,DENGUE SIN SEÑALES DE ALARMA
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,DENGUE SIN SEÑALES DE ALARMA
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,DENGUE SIN SEÑALES DE ALARMA
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,DENGUE SIN SEÑALES DE ALARMA
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,DENGUE SIN SEÑALES DE ALARMA


As we know there were issues with text, let's check department values:

In [20]:
dengueSub.departamento.value_counts()

departamento
PIURA            126410
LORETO            92496
UCAYALI           43707
MADRE DE DIOS     30562
TUMBES            27314
SAN MARTIN        26759
LA LIBERTAD       24377
ICA               21324
CAJAMARCA         19409
JUNIN             19199
LAMBAYEQUE        15618
AMAZONAS          12017
CUSCO             10476
HUANUCO            9046
AYACUCHO           8889
ANCASH             7214
LIMA               4255
PASCO              1963
\N                  444
PUNO                179
CALLAO               26
AREQUIPA              7
MOQUEGUA              1
Name: count, dtype: int64

In [21]:
dengueSub[dengueSub.departamento.str.contains('\\N',regex=False)]

,departamento,provincia,distrito,ano,semana,sexo,enfermedad
79131,\N,\N,\N,2009,1,F,DENGUE SIN SEÑALES DE ALARMA
79656,\N,\N,\N,2009,7,M,DENGUE SIN SEÑALES DE ALARMA
80500,\N,\N,\N,2009,26,F,DENGUE SIN SEÑALES DE ALARMA
84253,\N,\N,\N,2009,28,M,DENGUE SIN SEÑALES DE ALARMA
85102,\N,\N,\N,2009,4,F,DENGUE SIN SEÑALES DE ALARMA
...,...,...,...,...,...,...,...
480825,\N,\N,\N,2022,23,F,DENGUE SIN SEÑALES DE ALARMA
486719,\N,\N,\N,2022,33,F,DENGUE SIN SEÑALES DE ALARMA
487630,\N,\N,\N,2022,34,M,DENGUE SIN SEÑALES DE ALARMA
491395,\N,\N,\N,2022,43,F,DENGUE SIN SEÑALES DE ALARMA


In [22]:
dengueSub[dengueSub.distrito.str.contains('\\N',regex=False)]

,departamento,provincia,distrito,ano,semana,sexo,enfermedad
79131,\N,\N,\N,2009,1,F,DENGUE SIN SEÑALES DE ALARMA
79656,\N,\N,\N,2009,7,M,DENGUE SIN SEÑALES DE ALARMA
80500,\N,\N,\N,2009,26,F,DENGUE SIN SEÑALES DE ALARMA
84253,\N,\N,\N,2009,28,M,DENGUE SIN SEÑALES DE ALARMA
85102,\N,\N,\N,2009,4,F,DENGUE SIN SEÑALES DE ALARMA
...,...,...,...,...,...,...,...
480825,\N,\N,\N,2022,23,F,DENGUE SIN SEÑALES DE ALARMA
486719,\N,\N,\N,2022,33,F,DENGUE SIN SEÑALES DE ALARMA
487630,\N,\N,\N,2022,34,M,DENGUE SIN SEÑALES DE ALARMA
491395,\N,\N,\N,2022,43,F,DENGUE SIN SEÑALES DE ALARMA


Now, we keep the complete data:

In [23]:
# Let's replace that cell value in all the data frame:  
dengueSub.replace('\\N',None,regex=False,inplace=True)
# dropping
dengueSub.dropna(how='any',inplace=True,ignore_index=True)

/var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/ipykernel_33850/3813897578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengueSub.replace('\\N',None,regex=False,inplace=True)
/var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/ipykernel_33850/3813897578.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dengueSub.dropna(how='any',inplace=True,ignore_index=True)


In [24]:
dengueSub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501248 entries, 0 to 501247
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   departamento  501248 non-null  object
 1   provincia     501248 non-null  object
 2   distrito      501248 non-null  object
 3   ano           501248 non-null  int64 
 4   semana        501248 non-null  int64 
 5   sexo          501248 non-null  object
 6   enfermedad    501248 non-null  object
dtypes: int64(2), object(5)
memory usage: 26.8+ MB


The data is about people, but since there is no identifier for a person, it is possible that rows are repeated:

In [25]:
dengueSub[dengueSub.duplicated(keep=False)].sort_values(by=['distrito','semana','sexo'])

,departamento,provincia,distrito,ano,semana,sexo,enfermedad
225173,SAN MARTIN,EL DORADO,AGUA BLANCA,2014,8,F,DENGUE SIN SEÑALES DE ALARMA
225176,SAN MARTIN,EL DORADO,AGUA BLANCA,2014,8,F,DENGUE SIN SEÑALES DE ALARMA
225178,SAN MARTIN,EL DORADO,AGUA BLANCA,2014,8,F,DENGUE SIN SEÑALES DE ALARMA
225171,SAN MARTIN,EL DORADO,AGUA BLANCA,2014,8,M,DENGUE SIN SEÑALES DE ALARMA
225172,SAN MARTIN,EL DORADO,AGUA BLANCA,2014,8,M,DENGUE SIN SEÑALES DE ALARMA
...,...,...,...,...,...,...,...
226693,TUMBES,CONTRALMIRANTE VILLAR,ZORRITOS,2014,53,M,DENGUE SIN SEÑALES DE ALARMA
226701,TUMBES,CONTRALMIRANTE VILLAR,ZORRITOS,2014,53,M,DENGUE CON SEÑALES DE ALARMA
226702,TUMBES,CONTRALMIRANTE VILLAR,ZORRITOS,2014,53,M,DENGUE CON SEÑALES DE ALARMA
227559,TUMBES,CONTRALMIRANTE VILLAR,ZORRITOS,2014,53,M,DENGUE SIN SEÑALES DE ALARMA


To ease the count, we could add a column of ones:

In [26]:
dengueSub=dengueSub.assign(case=1)
dengueSub.head()

,departamento,provincia,distrito,ano,semana,sexo,enfermedad,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,DENGUE SIN SEÑALES DE ALARMA,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,DENGUE SIN SEÑALES DE ALARMA,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,DENGUE SIN SEÑALES DE ALARMA,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,DENGUE SIN SEÑALES DE ALARMA,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,DENGUE SIN SEÑALES DE ALARMA,1


In [27]:
dengueSub.enfermedad.value_counts()

enfermedad
DENGUE SIN SEÑALES DE ALARMA    444005
DENGUE CON SEÑALES DE ALARMA     54984
DENGUE GRAVE                      2259
Name: count, dtype: int64

In [28]:
dengueSub['enfermedad']=dengueSub.enfermedad.str.replace("DENGUE ","")
dengueSub['enfermedad']=dengueSub.enfermedad.str.replace("\s","_",regex=True)
dengueSub['enfermedad']

0         SIN_SEÑALES_DE_ALARMA
1         SIN_SEÑALES_DE_ALARMA
2         SIN_SEÑALES_DE_ALARMA
3         SIN_SEÑALES_DE_ALARMA
4         SIN_SEÑALES_DE_ALARMA
                  ...          
501243    SIN_SEÑALES_DE_ALARMA
501244    SIN_SEÑALES_DE_ALARMA
501245    SIN_SEÑALES_DE_ALARMA
501246    SIN_SEÑALES_DE_ALARMA
501247    SIN_SEÑALES_DE_ALARMA
Name: enfermedad, Length: 501248, dtype: object

In [29]:
# Let's save the current dataframe
dengueSub.to_csv("dengueSub.csv",index=False)

At this stage, we should aggregate the data:

In [ ]:
CasesByWeek=dengueSub.groupby(['departamento', 'provincia', 'distrito','ano', 'semana','sexo','enfermedad']).agg({'case': ['sum']})
CasesByWeek

We may turn that multi-index structure in a simpler way:

In [ ]:
CasesByWeek.columns=['cases'] # new name for the only column

CasesByWeek.reset_index(drop=False,inplace=True)

CasesByWeek

## Reshaping

### From Long to Wide

The object *CasesByWeek* shows the values in one column, and the other column serve as identifier (an index). Let's make a simple wide version (one index):

In [ ]:
CasesByWeek.pivot_table(values='cases',
                            index=['departamento'],
                            columns='enfermedad',aggfunc="sum")

The reshaping with two keys:

In [ ]:
CasesByWeek.pivot_table(values='cases',
                            index=['departamento', 'provincia'],
                            columns='enfermedad',aggfunc="sum")

The reshaping with two keys and two multi columns:

In [ ]:
CasesByWeek.pivot_table(values='cases',
                            index=['departamento', 'provincia'],
                            columns=['enfermedad','sexo'],aggfunc="sum")

Have you noticed that the more keys the more missing values?

Let's keep this last one, and it in a traditional way:

In [ ]:
CasesByWeek_Wide=CasesByWeek.pivot_table(values='cases',
                            index=['departamento', 'provincia'],
                            columns=['enfermedad','sexo'],aggfunc="sum").reset_index(drop=False)
CasesByWeek_Wide

We have multi index, let's flatten them:

In [ ]:
CasesByWeek_Wide.columns

In [ ]:
CasesByWeek_Wide.columns.name = None 

Now, concatenate the tuples:

In [ ]:
["_".join(pair) for pair in CasesByWeek_Wide.columns[2:]]

In [ ]:
# create the newNames
newNames=['departamento','provincia']
newNames.extend(["_".join(pair) for pair in CasesByWeek_Wide.columns[2:]])
newNames

In [ ]:
# renaming
CasesByWeek_Wide.columns=newNames
CasesByWeek_Wide.columns

We could re format the strings in the columns:

In [ ]:
CasesByWeek_Wide.columns.str.title().str.replace('\\s','',regex=True).str.replace("Dengue","",regex=False)

In [ ]:
# last step 
CasesByWeek_Wide.columns=CasesByWeek_Wide.columns.str.title().str.replace('\\s','',regex=True).str.replace("Dengue","",regex=False)

CasesByWeek_Wide

### Wide to Long

We should be able to transfor this wide version into a long one:

In [ ]:
# maybe not this one:
CasesByWeek_Wide.set_index('Departamento').stack().reset_index()

In [ ]:
CasesByWeek_Long=CasesByWeek_Wide.set_index(['Departamento','Provincia']).stack().reset_index()
CasesByWeek_Long

In [ ]:
CasesByWeek_Long.rename(columns={'level_2':'status',0:'cases'},inplace=True)
CasesByWeek_Long